## 카운트 기반의 단어 표현(Count based Word Representation)

### bag of words (BoW)

In [2]:
#1. 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.

from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
    # 온점 제거 및 형태소 분석
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)
    
    word_to_index = {}
    bow = []
    
    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            # BoW에 전부 기본값 1을 넣는다
            bow.insert(len(word_to_index)-1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더함
            bow[index] = bow[index] + 1
            
    return word_to_index, bow

In [3]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary: ', vocab)
print('bag of words vector: ', bow)

vocabulary:  {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector:  [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [4]:
# 문서2 : 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.
doc2 = "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다."
vocab, bow = build_bag_of_words(doc2)
print('vocabulary: ', vocab)
print('bag of words vector: ', bow)

vocabulary:  {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector:  [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [5]:
# 문서3 = 문서1 + 문서2
doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)
print('vocabulary: ', vocab)
print('bag of words vector: ', bow)

vocabulary:  {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector:  [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


#### CountVectorizer 클래스

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector: ', vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지 출력
print('vocabulary: ', vector.vocabulary_)

bag of words vector:  [[1 1 2 1 2 1]]
vocabulary:  {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


'I' 같은 경우 CountVectorizer가 기본적으로 길이가 2 이상인 문자에 대해서만 토큰으로 인식하기 때문에 BoW를 만드는 과정에서 사라짐

#### 불용어 제거한 BoW

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

사용자가 직접 정의한 불용어 사용

In [9]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print('bag of words vector: ', vect.fit_transform(text).toarray())
print('vocabulary: ', vect.vocabulary_)

bag of words vector:  [[1 1 1 1 1]]
vocabulary:  {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


CountVectorizer에서 제공하는 자체 불용어 사용

In [10]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words='english')
print('bag of words vector: ', vect.fit_transform(text).toarray())
print('vocabulary: ', vect.vocabulary_)

bag of words vector:  [[1 1 1]]
vocabulary:  {'family': 0, 'important': 1, 'thing': 2}


NLTK에서 지원하는 불용어 사용

In [12]:
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector: ', vect.fit_transform(text).toarray())
print('vocabulary: ', vect.vocabulary_)

bag of words vector:  [[1 1 1 1]]
vocabulary:  {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


### TF-IDF(Term Frequency-Inverse Document Frequency)

In [14]:
import pandas as pd
from math import log

docs = [
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [16]:
# 총 문서의 수
N = len(docs)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t, d) * idf(t)

In [18]:
# DTM (tf)
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))
        
        
tf_ = pd.DataFrame(result, columns=vocab)

In [19]:
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [20]:
# IDF
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))
    
    
idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [21]:
# TF-IDF
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))
        
tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


#### 사이킷런을 이용한 DTM과 TF-IDF

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do'
]

vector = CountVectorizer()


# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [23]:
# TF-IDF를 자동 계산해주는 TfidfVectorizer (조정된 식 사용)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
